In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
#! ls

In [ ]:
def get_fileNames(filePath):
    """
    Fetch the names of all files in the folder, and returns a list of all the .csv files.
    """ 
    fileList = os.listdir(filePath)
    fileNames = sorted([file for file in fileList if file.endswith('.csv')])
    
    return fileNames

In [ ]:
def get_XYZ(filePath, fileName):
    """
    Takes a .csv file and reads it into a Pandas DataFrame. Returns 2D arrays of x,y coordinates X and Y along
    with the contour data Z
    """
    # Import data into Dataframe and transpose so that excitation data (x-axis) is arranged in to columns
    # and emission data (y-axis) is arranged into rows.
    Z = pd.read_csv(os.path.join(filePath, fileName), header=0, index_col=0).T
    
    # Set x- and y-axis values
    x = Z.columns
    y = Z.index.astype(int)
    # Set X and Y (2D arrays of the x and y points)
    X, Y = np.meshgrid(x, y)
    #print(X.shape, Y.shape)
    
    return X, Y, Z

In [ ]:
def plot_contour(X, Y, Z, fileName, outFormat='png'):
    """
    Plots the Excitation and Emission data and saves the fig as a .png file.
    """
    hfont = {'fontname':'Arial'}

    fig, ax = plt.subplots(1, 1)
    cs = ax.contourf(X, Y, Z, 100, cmap='jet')
    
    plt.rcParams.update({'font.size': 18})
    #ax.set_title(os.path.basename(f'{fileName[:-4]}'), **hfont)
    ax.set_xlabel('Emission Wavelength (nm)', **hfont) 
    ax.set_ylabel('Excitation Wavelength (nm)', **hfont)

    plt.xticks(**hfont)
    plt.yticks(**hfont)
    
    plt.tight_layout()
    
    # Configure colorbar scale
    maxZ = round(max(Z.max()), -2) # max intensity value
    cbar = fig.colorbar(cs, spacing = 'proportional')
    cbar.set_ticks(np.arange(0, maxZ+1, maxZ/5))
    
    outfile = os.path.join(filePath, fileName[:-4])
    fig.savefig(f'{outfile}.{outFormat}', dpi=300, bbox_inches='tight', transparent=True)
    
    return

In [ ]:
filePath = input('Enter path to data folder: ')
outFormat = input('Enter output file type: ')

fileNames = get_fileNames(filePath)

print(f'There are {len(fileNames)} to parse.\nFile list = {fileNames}')

count = 0
for file in fileNames:
    try:
        X, Y, Z = get_XYZ(filePath, file)
        plot_contour(X, Y, Z, file, outFormat)
        count += 1
    except:
        print(f'Failed to plot {file}.')
        continue
    
print(f'Succesfully plotted {count}/{len(fileNames)} files.')